<a href="https://colab.research.google.com/github/DivyaRoopa123/Intern-week--1/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import files

uploaded = files.upload()


Saving True.csv to True (5).csv
Saving Fake.csv to Fake (5).csv


In [26]:

import re
from sklearn.model_selection import train_test_split
print("Columns in df:", df.columns.tolist())

possible_text_cols = ["clean_text", "text_clean", "text", "content", "headline", "body"]
text_col = None
for c in possible_text_cols:
    if c in df.columns:
        text_col = c
        break


possible_label_cols = ["label", "Label", "target", "class"]
label_col = None
for c in possible_label_cols:
    if c in df.columns:
        label_col = c
        break


if text_col is None:
    str_cols = [c for c in df.columns if df[c].dtype == object]
    if len(str_cols) == 0:
        raise ValueError("No text-like columns in dataframe. Please provide 'text' or upload correct CSV.")
    print("No standard text column found. Concatenating string columns:", str_cols)
    df["clean_text"] = df[str_cols].astype(str).apply(lambda r: " ".join(r.values), axis=1)
    text_col = "clean_text"


def clean_text_simple(t):
    t = str(t)
    t = t.lower()
    t = re.sub(r"http\S+|www\.\S+", " ", t)
    t = re.sub(r"[^a-z0-9\s]", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

if text_col != "clean_text":
    print(f"Creating 'clean_text' from '{text_col}'")
    df["clean_text"] = df[text_col].fillna("").apply(clean_text_simple)
else:

    df["clean_text"] = df["clean_text"].fillna("").apply(clean_text_simple)


if label_col is None:

    if "label" in df.columns:
        label_col = "label"
    else:
        raise KeyError("No label column found (tried: {}). Make sure your dataframe has a label column.".format(possible_label_cols))


y_raw = df[label_col]

if y_raw.dtype == object:
    unique_vals = list(y_raw.dropna().unique())
    print("Label unique values (strings):", unique_vals[:10])

    mapping = {}
    if set(map(str.lower, unique_vals)) <= {"fake", "false", "0"} or "fake" in map(str.lower, unique_vals):
        mapping = {v: (0 if v.lower() in ("fake","false","0") else 1) for v in unique_vals}
    elif set(map(str.lower, unique_vals)) <= {"true","real","1"} or "true" in map(str.lower, unique_vals):
        mapping = {v: (1 if v.lower() in ("true","real","1") else 0) for v in unique_vals}
    else:

        mapping = {v: i for i,v in enumerate(sorted(unique_vals))}
    print("Mapping labels:", mapping)
    df["label_numeric"] = y_raw.map(mapping).astype(int)
else:

    df["label_numeric"] = y_raw.fillna(0).astype(int)


y = df["label_numeric"].values
unique_classes = np.unique(y)
print("Classes in dataset after normalization:", unique_classes)

if len(unique_classes) < 2:

    print("Warning: Only one class present. Splitting without stratify.")
    stratify_arg = None
else:
    stratify_arg = y


X = df["clean_text"].values
try:
    if stratify_arg is None:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=stratify_arg)
    print("Train size:", len(X_train), "Test size:", len(X_test))
except Exception as e:

    print("Error during train_test_split:", type(e), e)
    print("Shapes: X:", X.shape, "y:", y.shape)
    raise




Columns in df: ['text', 'label']
Creating 'clean_text' from 'text'
Classes in dataset after normalization: [0 1]
Train size: 35918 Test size: 8980
